# SFM Forecasting Algorithm Simulation

This notebook simulates the **LSTM**, **RNN**, and **Dense** neural network algorithms used in the SFM (Smart Fish Management) system for time-series forecasting.

The implementation mirrors the TensorFlow.js version used in the web application.

## 1. Setup and Imports

In [ ]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, SimpleRNN, Dense, Dropout, Flatten

print(f"TensorFlow version: {tf.__version__}")

## 2. Data Preparation

Functions to normalize data and create sequences for training.

In [ ]:
def normalize_data(data):
    """Normalize data to 0-1 range"""
    min_val = np.min(data)
    max_val = np.max(data)
    range_val = max_val - min_val
    normalized = (data - min_val) / range_val if range_val > 0 else np.zeros_like(data)
    return normalized, min_val, max_val

def denormalize_data(normalized, min_val, max_val):
    """Denormalize data back to original range"""
    range_val = max_val - min_val
    return normalized * range_val + min_val

def create_sequences(data, sequence_length):
    """Create input sequences and targets for training"""
    xs, ys = [], []
    for i in range(len(data) - sequence_length):
        xs.append(data[i:i + sequence_length])
        ys.append(data[i + sequence_length])
    return np.array(xs), np.array(ys)

## 3. Model Architectures

### 3.1 LSTM (Long Short-Term Memory)
- Best for complex time series with long-term dependencies
- 2 LSTM layers (50 units each), Dropout (0.2)
- Highest accuracy, slower training

In [ ]:
def build_lstm_model(sequence_length, learning_rate=0.001):
    model = Sequential([
        LSTM(50, return_sequences=True, input_shape=(sequence_length, 1)),
        Dropout(0.2),
        LSTM(50, return_sequences=False),
        Dropout(0.2),
        Dense(1)
    ])
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate),
                  loss='mse', metrics=['mae'])
    return model

### 3.2 RNN (Simple Recurrent Neural Network)
- Balanced accuracy and speed
- 2 SimpleRNN layers (32 units each), Dropout (0.2)

In [ ]:
def build_rnn_model(sequence_length, learning_rate=0.001):
    model = Sequential([
        SimpleRNN(32, return_sequences=True, input_shape=(sequence_length, 1)),
        Dropout(0.2),
        SimpleRNN(32, return_sequences=False),
        Dropout(0.2),
        Dense(1)
    ])
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate),
                  loss='mse', metrics=['mae'])
    return model

### 3.3 Dense (Multi-layer Perceptron)
- Fastest training, good for simple patterns
- Flatten + 3 Dense layers (128, 64, 32), Dropout (0.3)

In [ ]:
def build_dense_model(sequence_length, learning_rate=0.001):
    model = Sequential([
        Flatten(input_shape=(sequence_length, 1)),
        Dense(128, activation='relu'),
        Dropout(0.3),
        Dense(64, activation='relu'),
        Dropout(0.3),
        Dense(32, activation='relu'),
        Dense(1)
    ])
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate),
                  loss='mse', metrics=['mae'])
    return model

## 4. Training and Forecasting

In [ ]:
def forecast(data, algorithm='lstm', sequence_length=10, epochs=50, prediction_steps=7):
    """Train and forecast using selected algorithm"""
    if algorithm == 'lstm':
        model = build_lstm_model(sequence_length)
    elif algorithm == 'rnn':
        model = build_rnn_model(sequence_length)
    elif algorithm == 'dense':
        model = build_dense_model(sequence_length)
    else:
        raise ValueError(f"Unknown algorithm: {algorithm}")
    
    # Normalize and create sequences
    normalized, min_val, max_val = normalize_data(data)
    xs, ys = create_sequences(normalized, sequence_length)
    xs = xs.reshape(-1, sequence_length, 1)
    ys = ys.reshape(-1, 1)
    
    # Train
    model.fit(xs, ys, epochs=epochs, batch_size=32, validation_split=0.1, verbose=0)
    
    # Predict
    predictions = []
    current_seq = normalized[-sequence_length:].copy()
    for _ in range(prediction_steps):
        X = current_seq.reshape(1, sequence_length, 1)
        pred = model.predict(X, verbose=0)[0, 0]
        predictions.append(pred)
        current_seq = np.append(current_seq[1:], pred)
    
    predictions = denormalize_data(np.array(predictions), min_val, max_val)
    return predictions

## 5. Run Simulation with Sample Data

Simulating fish weight growth over time (sample data).

In [ ]:
# Sample historical data (e.g., average fish weight in grams over weeks)
historical_data = np.array([10, 12, 15, 18, 22, 25, 30, 35, 40, 45, 50, 55, 60, 68, 75, 82, 90, 98, 108, 118])

# Run forecast with LSTM
lstm_predictions = forecast(historical_data, algorithm='lstm', epochs=30, prediction_steps=7)
print("LSTM Predictions:", lstm_predictions.round(2))

# Run forecast with RNN
rnn_predictions = forecast(historical_data, algorithm='rnn', epochs=30, prediction_steps=7)
print("RNN Predictions:", rnn_predictions.round(2))

# Run forecast with Dense
dense_predictions = forecast(historical_data, algorithm='dense', epochs=30, prediction_steps=7)
print("Dense Predictions:", dense_predictions.round(2))

## 6. Visualize Results

In [ ]:
plt.figure(figsize=(12, 6))
x_hist = np.arange(len(historical_data))
x_pred = np.arange(len(historical_data), len(historical_data) + 7)

plt.plot(x_hist, historical_data, 'b-o', label='Historical Data', markersize=4)
plt.plot(x_pred, lstm_predictions, 'r--s', label='LSTM Forecast', markersize=5)
plt.plot(x_pred, rnn_predictions, 'g--^', label='RNN Forecast', markersize=5)
plt.plot(x_pred, dense_predictions, 'm--d', label='Dense Forecast', markersize=5)

plt.xlabel('Time Step')
plt.ylabel('Value')
plt.title('SFM Forecasting Algorithm Comparison')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()